# Load data

In [73]:
import pandas as pd

# existing_file_path = 'data/input/test_input.xlsx'
# existing_file_path = 'data/input/test_input_minimized.xlsx'
existing_file_path = 'data/input/test_data_table.xlsx'

df = pd.read_excel(existing_file_path)

# Reading data with pandas

In [74]:
# Read the entire dataframe
# df

## Read headers
# print(df.columns)

## Read individual columns
# print(df.Date)
#  or
# print(df['Net Donation'])
#  or
# print(df[['Date', "Description", "Net Donation"]])

## Read top 3 rows
# print(df.head(3))

## Read row 3 (which has an index of 2),
# print(df.iloc[2])
# then 1-5 (which is index 0-4)
# print(df.iloc[0:5])
# Read specific location (R, C)
# print(df.iloc[2, 1]) # 3rd row, 2nd column

## Iterate over rows
# for index, row in df.iterrows():
    # print(index, row['Date'], row['Description'])
    # print(index, row)
    # print(index, row.Date)

# Conditional selection of rows
# df.loc[df['Net Donation'] > 60]

# Generate statistics
# df.describe()


# Sorting data

In [80]:
# df # Before sorting

# Sorting by single columns
# df = df.sort_values(by=['Description'])
# df = df.sort_values(by=['Event'])
# df = df.sort_values(by=['Source Title'])

# Sorting can be done on multiple columns with this one line of code
df = df.sort_values(by=['Source Title','Event','Description'])

df # After sorting

,Date,Description,Net Donation,Stripe Fee,Platform Fee,Total Gross Donation,Email,Event,Source Title
6,2024-03-07,follylollycontact@gmail.com,775.00,NaN,NaN,775.00,follylollycontact@gmail.com,2024 Twin Cities Pride Festival,2024 Booth Vendor - Pride Festival
5,2024-03-07,reese@joycoast.com,100.00,NaN,NaN,100.00,reese@joycoast.com,2024 Twin Cities Pride Festival,2024 Booth Vendor - Pride Festival
1,2024-03-08,sheila.sylvander@hennepintech.edu,50.00,NaN,NaN,50.00,sheila.sylvander@hennepintech.edu,2024 Twin Cities Pride Festival,2024 Booth Vendor - Pride Festival
0,2024-03-08,lisa.determan@pentair.com,50.00,NaN,NaN,50.00,lisa.determan@pentair.com,2024 Twin Cities Pride Festival,2024 TC Pride March Application
2,2024-03-08,madisen@meals-on-wheels.com,50.00,NaN,NaN,50.00,madisen@meals-on-wheels.com,2024 Twin Cities Pride Festival,2024 TC Pride March Application
4,2024-03-08,Subscription update,30.16,1.24,0.9,32.30,NaN,NaN,NaN
3,2024-03-08,Twin Cities Pride - Order 42731,102.06,3.36,NaN,105.42,NaN,NaN,NaN


# Sum revenue into different categories

In [79]:
categoryDictionary = {}

for index, row in df.iterrows():
  if row['Source Title'] not in categoryDictionary:
    categoryDictionary[row['Source Title']] = []
  categoryDictionary[row['Source Title']].append(row['Event'])
# print as a json object
print(categoryDictionary)

{'2024 Booth Vendor - Pride Festival': ['2024 Twin Cities Pride Festival', '2024 Twin Cities Pride Festival', '2024 Twin Cities Pride Festival'], '2024 TC Pride March Application': ['2024 Twin Cities Pride Festival', '2024 Twin Cities Pride Festival'], nan: [nan, nan]}


# Convert the dataframe data back into its original form


In [ ]:
# Date conversions might be needed for writing to excel files
# df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)

# Description
# REFUND FOR PAYMENT
# One-time donation
# Harness custom donation

# Event
# 2024 Twin Cities Pride Festival	

# Source Title
# 2024 Booth Vendor - Pride Festival
# 2024 Food Vendor - Pride Festival
# 2024 Queer Writes Book Fair @ Pride
# 2024 TC Pride March Application

# Write to Excel

In [ ]:
# df.to_excel('data/output/test_output.xlsx', index=False)